# Defining your own type


Types can be thought as "boxes" containing data. Each "box" is called  **``field``** and can contain any  **``Type``** inside, each field is identified by a name filed. In order to see the different fields of a type you can use the function **``fieldnames``**.

- Each field can have a type, the symbol **`::`** is used to assign a type to a fieldname.
    - For example **`x::Float64`** states that `x` has type ``Float64``.
- Each field can have a different type

In [377]:
workspace()
type Point
    x::Float64
    y::Float64
    z::Float64
end
p = Point(10,10,2)

Point(10.0,10.0,2.0)

In [378]:
fieldnames(p)

3-element Array{Symbol,1}:
 :x
 :y
 :z

In [379]:
p.x = 23

23

In [380]:
p

Point(23.0,10.0,2.0)

## Inmutable Types

As the name suggests, you cannot mutate the fieldnames of **``immutable``** types.


**An immutable object might contain mutable objects**, such as arrays, as fields. Those contained objects will remain mutable; only the fields of the immutable object itself cannot be changed to point to different objects.

In [361]:
workspace()
immutable Point
    x::Float64
    y::Float64
    z::Float64
end
p = Point(10,10,2)

Point(10.0,10.0,2.0)

In [362]:
p.x = 23

LoadError: type Point is immutable

In [363]:
workspace()
immutable VectorPoint
    x::Array{Float64}
    y::Array{Float64}
    z::Array{Float64}
end
p = VectorPoint([10,2],[10,1],[10,10])

VectorPoint([10.0,2.0],[10.0,1.0],[10.0,10.0])

In [367]:
p.x = [2.,3.]

LoadError: type VectorPoint is immutable

In [706]:
immutable Family
    members::Array{String}
    name::String
end

In [707]:
simpsons = Family(["Lisa","Hommer","Bart","Marge"],"Simpson")

Family(String["Lisa","Hommer","Bart","Marge"],"Simpson")

In [708]:
push!(simpsons.members, "maggie")

5-element Array{String,1}:
 "Lisa"  
 "Hommer"
 "Bart"  
 "Marge" 
 "maggie"

In [371]:
simpsons

Family(String["Lisa","Hommer","Bart","Marge","maggie"],"Simpson")

In [710]:
immutable Model
    name::String
    Θ::Array
end

In [711]:
perceptron = Model("Percepton", zeros(10))

Model("Percepton",[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])

In [712]:
perceptron.Θ[1] = 12

12

In [713]:
perceptron.Θ'

1×10 Array{Float64,2}:
 12.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [714]:
perceptron.Θ = perceptron.Θ + 1

LoadError: type Model is immutable

In [715]:
perceptron.Θ .= perceptron.Θ + 1

10-element Array{Float64,1}:
 13.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0

## Parametric types

To make the code more general we can use ** parametric types** which define a family of types by a parameter. 

- Parametric types can depend on multiple parameters `Point{T1,T2}`.


In [310]:
workspace()
type Point{T}
    x::T
    y::T
    z::T
end

In [311]:
p = Point(10,10,2)

Point{Int64}(10,10,2)

In [312]:
p = Point(10.,10.,2.)

Point{Float64}(10.0,10.0,2.0)

#### This code
``` Julia
type Point{T}
    x::T
    y::T
    z::T
end
```

#### Already defines all this


``` Julia
type Point{T<:Real}
  x::T
  y::T

  Point(x,y,z) = new(x,y,z)
end

Point{T<:Real}(x::T, y::T, z::T) = Point{T}(x,y,z)
```

In [208]:
workspace()
type Point{T1,T2}
    x::T1
    y::T1
    z::T2
end

In [210]:
p = Point(10.,10.,2)

Point{Float64,Int64}(10.0,10.0,2)

## Set restrictions on custom types


Sometimes we don't want to have types that are too general and can be missused in unnexpected ways. In order to avoid this we can put restrictions. For example, our `Point` type can be created to be used only with numbers inside its fieldnames, nevertheless, our previous definition

``` Julia
type Point{T}
    x::T
    y::T
    z::T
end
```

can accept any `Type` inside `x`, `y` and `z` as long as it's the same type for all fieldnames. 

``` Julia
Point("This", "is", "unnexpected")
```
```
Point{String}("This","is","unnexpected")
```

#### Using the subtype operator <:

The operator **`<:`** allows us to verify if a `Type` is subtype of an `AbstractType`.


In [260]:
Int64 <: Real

true

In [261]:
workspace()
type Point{T<:Real}
    x::T
    y::T
    z::T
end

In [262]:
Point("This", "is", "unnexpected")

LoadError: MethodError: no method matching Point{T<:Real}(::String, ::String, ::String)[0m
Closest candidates are:
  Point{T<:Real}{T}(::Any) at sysimg.jl:53[0m

In [264]:
p = Point(10,10,2)

Point{Int64}(10,10,2)

## Set default values for a custom type

Constructors can be used to generate ways to construct custom types. This could be usefull, for example, in order to set default values to your custom types.

In [860]:
workspace()
type Point{T}
    x::T
    y::T
    z::T
end

# Make a constructor that sets some default values 
Point(x,y) = Point(x,y,0)

# Make a constructor that sets all values as default
Point() = Point(0,0,0)

Point{T}

In [861]:
Point(10,10)

Point{Int64}(10,10,0)

In [862]:
Point()

Point{Int64}(0,0,0)

## Set restrictions for custom types, inner constructors

If we want to enforce properties of a type in its construction we can use inner constructors with restriction.

We can add a function inside a type used to instanciate the type.

This function can be used for example in order to..

- Put restrictions on the type, such as..
     - not allow negative values 
     - Not allow 'rare' characters

In [869]:
workspace()
type OrderedPair
  x::Real
  y::Real
  OrderedPair(x,y) = x > y ? error("out of order") : new(x,y)
end

In [870]:
OrderedPair(1,10)

OrderedPair(1,10)

In [871]:
OrderedPair(10,1)

LoadError: out of order

## Parametric types with restrictions
#### inner constructor for parametric types => outer constructor needed

In [885]:
workspace()
type OrderedPair{T}
  x::T
  y::T 
  OrderedPair(x,y) = x > y ? error("out of order") : new(x,y)
end

# THIS WILL NOT WORK
OrderedPair(1,10)

LoadError: MethodError: no method matching OrderedPair{T}(::Int64, ::Int64)[0m
Closest candidates are:
  OrderedPair{T}{T}(::Any) at sysimg.jl:53[0m

In [882]:
workspace()
type OrderedPair{T}
  x::T
  y::T 
  OrderedPair(x,y) = x > y ? error("out of order") : new(x,y)
end

# Need outer constructor
OrderedPair{T}(x::T,y::T) = OrderedPair{T}(x,y) 

OrderedPair{T}

In [881]:
OrderedPair(1,10)

OrderedPair{Int64}(1,10)

In [916]:
OrderedPair(100,10)

LoadError: out of order

#### You can have default values and parametric restricted types

In [60]:
workspace()
type OrderedPair{T}
  x::T
  y::T 
  OrderedPair(x,y) = x > y ? error("out of order") : new(x,y)
end

# Need outer constructor
OrderedPair{T}(x::T,y::T) = OrderedPair{T}(x,y) 
OrderedPair{T}(x::T) = OrderedPair{T}(x,x) 


OrderedPair{T}

In [61]:
OrderedPair(1)

OrderedPair{Int64}(1,1)

## Make custom print for a custom type

In [829]:
import Base.show

In [830]:
type Point
       x::Int
       y::Int
end

In [831]:
Base.show(io::IO, p::Point) = print(io, "Point: x=$(p.x), y=$(p.y)")

In [97]:
Point(4,5)

LoadError: UndefVarError: Point not defined

# For another time

## Easy default parameters with Parameters package

In [102]:

#Pkg.add("Parameters")
using Parameters

In [103]:
@with_kw type A
           a::Int = 6
           b::Float64 = -1.1
           c::UInt64
       end


A

In [104]:
a = A(c=4)

A
  a: Int64 6
  b: Float64 -1.1
  c: UInt64 4


In [107]:
a.c

0x0000000000000004

### Parameterized type with default values not working

In [121]:
workspace()
type PosPoint{T}
       x::T
       y::T
    PosPoint(x,y) = x < 0 || y < 0 ? error("coordinates must be positive") : new(x,y)
end

PosPoint{T}(x::T,y::T) = PosPoint{T}(x,y) 
#PosPoint() = PosPoint{T}(zero(T),zero(T))  # Not working
PosPoint() = PosPoint(0,0)  # Not working



PosPoint{T}

In [117]:
PosPoint(20,10)

PosPoint{Int64}(20,10)

In [118]:
typeof(zero(Float32))

Float32

In [119]:
PosPoint(20,-10)

LoadError: coordinates must be positive

In [122]:
PosPoint()

PosPoint{Int64}(0,0)

### Assigning  a default value to a type 

By default we might think that Giants are quite stupid and therefore we might be interested into assigning "stupid" intelligence as default.

Notice that default values in Julia are defined after ```;```

This can be specially handly in order to provide default values for types, for example when there could be 'standard values' for some of the fileds inside a type and we do not want to bother the user selecting field values. 

Using this trick we can allow the user to set values to all the fields the user wants to but at the same time, allow the user **not to** define all possible fields .

In [799]:
workspace()

In [833]:
type Giant{T}
    weight::T
    height::T
    intelligence::String
    
    function Giant(w, h ; intelligence="stupid")
        if h<2
            throw(ArgumentError("Giants are taller\n"))
        end
        return new(w, h ,  intelligence)
    end
    
end

## IMPORTANT TO ADD CONSTRUCTOR FOR EVERY TYPE T !!!!
Giant{T}(weight::T, height::T) = Giant{T}(weight, height)


LoadError: invalid redefinition of constant Giant

In [834]:
giant = Giant(3.,5.)

Giant{Float64}(3.0,5.0,"stupid")

In [836]:
giant.weight, giant.height, giant.intelligence

(3.0,5.0,"stupid")

In [837]:
giant.intelligence = "smart"

"smart"

In [838]:
giant.weight, giant.height, giant.intelligence

(3.0,5.0,"smart")

Notice that we cannot pass now a value for the field intelligence

In [839]:
rare_giant = Giant(5.,4.,"smart")

Giant{Float64}(5.0,4.0,"smart")

Notice that we cannot use this definition since we need to specify how to create a Giant  when we pass the intelligence field.

### Allowing types to have fields with defeault values

https://groups.google.com/forum/#!topic/julia-users/9jM7GIoh7YY

In [826]:
workspace()

type Giant{T}
    weight::T
    height::T
    intelligence::String
    
    function Giant(w, h ; intelligence="stupid")
        if h<2
            throw(ArgumentError("Giants are taller\n"))
        end
        return new(w, h, intelligence)
    end
    
    function Giant(w, h, intelligence)
        if h<2
            throw(ArgumentError("Giants are taller\n"))
        end
        return new(w, h, intelligence)
    end
end

## IMPORTANT TO ADD CONSTRUCTOR FOR EVERY TYPE T !!!!
Giant{T}(weight::T, height::T) = Giant{T}(weight, height)
Giant{T}(weight::T, height::T, intelligence::String) = Giant{T}(weight, height, intelligence)

rare_giant = Giant(5., 4., "smart")
giant = Giant(3., 5.)

Giant{T}

In [827]:
rare_giant = Giant(5., 4., "smart")

Giant{Float64}(5.0,4.0,"smart")

In [828]:
giant = Giant(3., 5.)

Giant{Float64}(3.0,5.0,"stupid")

#### Type man


The following example is the type man that contains 3 fields, **```heigh```**,**``` weight```** and **```name```**.

In [ ]:
fieldnames(david)

Notice that **you cannot specify a particular value on a field**.

In [ ]:
type girl{T}
    height::T
    weight::T
    name="Julia"
end

Nevertheless you can define a type and a (so called) inner constructor which might specify a particular value of the type.

In [ ]:
type girl2{T}
    height::T
    weight::T
    name::String
end

In [ ]:
girl2(23, 123, "julia")

In [ ]:
typeof(girl2)

In [ ]:
methodswith(girl2)

### Inner constructor

In [855]:
workspace()
type girl4{T}
    height::T
    weight::T
    name::String
    
    function girl4(height, weight)
        return new(height,weight,"NoName")
    end
end
girl4{T}(x::T, y::T)  = girl4(x,y)

girl4{T}

In [856]:
girl4(23,123)

LoadError: StackOverflowError:

# Making custom print for a defined type

Once defined a type we can import ```Base.show``` and define a ```show``` method for the defined type


In [ ]:
import Base.show

In [ ]:
type Point
       x::Int
       y::Int
end

In [ ]:
Base.show(io::IO, p::Point) = print(io, "Point: x=$(p.x), y=$(p.y)")

In [ ]:
#Base.print(io::IO, p::Point) = string(p)

In [ ]:
Point(4,5)

In [ ]:
print(Point(4,5))

In [ ]:
[Point(2,5), Point(1,1), Point(5,6)]

### Another example

In [ ]:
require("Enum")

using Calendar
   type LogMessage
     stamped::CalendarTime
     msgstat::MsgStatus
     message::String
end
   import Base.show
   show(m::LogMessage) =
   print("$(m.stamped): $(m.msgstat) >> $(m.message)")
   msg = LogMessage(Calendar.now(), WARN, "Be very afraid")
   show(msg)

### The importance of declaring code inside functions

In [857]:
@time begin
pos = 0
num_steps = 10^4
numwalkers = 10^4
final_square_positions = Int[]

for i in 1:numwalkers
    for j in 1:num_steps
        pos += ifelse( rand() < 0.5, -1, +1)
    end
        push!(final_square_positions, pos^2)
end
    
end

 18.365446 seconds (385.29 M allocations: 7.232 GB, 4.29% gc time)


In [858]:
function fool_function()
    pos = 0
    num_steps = 10^4
    numwalkers = 10^4
    final_square_positions = Int[]

    for i in 1:numwalkers
        for j in 1:num_steps
            pos += ifelse( rand() < 0.5, -1, +1)
        end
            push!(final_square_positions, pos^2)
    end
end

fool_function (generic function with 1 method)

In [859]:
@time fool_function()

  0.347206 seconds (4.89 k allocations: 473.591 KB)
